# Benders Decomposition - Multi-Cut
[Murwan Siddig](mailto:msiddig@clemson.edu)

-------------------------------------------------------------------------------------------------------------

* Consider the following two-stage stochastic program:

\begin{equation}
\begin{array}{llll}
\displaystyle \min_{x, y \, \in \, \mathbb{R}^2_+} & 3x_1 + 2x_2 - \mathbb{E}_{\xi}(15y_1 + 12y_2) \\
\mbox{s.t.} & 3y_1 + 2y_2 \leq x_1 \\
            & 2y_1 + 5y_2 + \leq x_2 \\
            & 0.8\xi_1 \leq y_1 \leq \xi_1\\
            & 0.8\xi_2 \leq y_2 \leq \xi_2\\
            & x,y \geq 0\\
\end{array}
\end{equation}

with 
\begin{equation}
(\xi_1,\xi_2) = \left\{
\begin{array}{llll}
(4,4) & \text{w.p} \; \frac{1}{4} \\
(4,8) & \text{w.p} \; \frac{1}{4} \\
(6,4) & \text{w.p} \; \frac{1}{4} \\
(6,8) & \text{w.p} \; \frac{1}{4} \\
\end{array}
\right.
\end{equation}


---------------------------------------------------------------------------------------------

* Subproblem **(optimality)**:
\begin{equation}
\begin{array}{llll}
\displaystyle Q(x,\xi^s) := \min_{y \, \in \, \mathbb{R}^2_+} & -(15y_1+12y_2) \\
\mbox{s.t.}   & 3y_1+2y_2\leq \hat{x}_1 & (\pi^s_1)\\
              & 2y_1+5y_2\leq \hat{x}_2 &(\pi^s_2)\\
              & y_1\geq 0.8\xi^s_{1} & (\pi^s_3)\\
              & y_2\geq 0.8\xi^s_{2} & (\pi^s_4)\\
              & y_1\leq \xi^s_{1} & (\pi^s_5)\\
              & y_2\leq \xi^s_{2} & (\pi^s_6)\\
              & y_1,y_2\geq 0 \\
\end{array}
\end{equation}


* Subproblem **(feasiblity)**:
\begin{equation}
\begin{array}{llll}
\displaystyle \min_{y \, \in \, \mathbb{R}^2_+} & \sum_{i=1}^{6}(v_i^++v_i^-)\\
\mbox{s.t.}   & 3y_1+2y_2+(v_1^+-v_1^-)\leq \hat{x}_1 & (\lambda^s_1)\\
              & 2y_1+5y_2+(v_2^+-v_2^-)\leq \hat{x}_2 & (\lambda^s_2)\\
              & y_1+(v_3^+-v_3^-)\geq 0.8\xi^s_{1} & (\lambda^s_3)\\
              & y_2+(v_4^+-v_4^-)\geq 0.8\xi^s_{2} & (\lambda^s_4)\\
              & y_1+(v_5^+-v_5^-)\leq \xi^s_{1} & (\lambda^s_5)\\
              & y_2+(v_6^+-v_6^-)\leq \xi^s_{2} & (\lambda^s_6)\\
              & y_1,y_2\geq 0 \\
              & v_i^+, v_i^-\geq 0, \forall i=1, \dots, 6\\
\end{array}
\end{equation}

* Master problem **(Multi cut)**:

\begin{equation}
\begin{array}{llll}
\displaystyle \min_{x, y \, \in \, \mathbb{R}^2_+} & 3x_1 + 2x_2 +\sum_{s=1}^{4} \theta_s\times p_s \\
\mbox{s.t.} & \theta^s \geq  \pi^s_1x_1+\pi^s_2x_2+\pi^s_3(0.8\xi^s_{1})+\pi^s_4(0.8\xi^s_{2})+\pi^s_5(\xi^s_{1})+\pi^s_6(\xi^s_{2}) & \forall \; \pi^s \in \Pi, s=1,\dots,4 \\
            & 0 \geq \lambda^s_1x_1+\lambda^s_2x_2+\lambda^s_3(0.8\xi^s_{1})+\lambda^s_4(0.8\xi^s_{2})+\lambda^s_5(\xi^s_{1})+\lambda^s_6(\xi^s_{2}) & \forall \; \lambda^s \in \Lambda, s=1,\dots,4 \\
    & 3x_1 + 2x_2 +\sum_{s=1}^{4} \theta_s\times p_s \geq \underline{z}_{\text{MVP}} \\
& x_1,x_2 \geq 0 \\
\end{array}
\end{equation}

where $\underline{z}_{\text{MVP}}$ is a lower bound obtained by solving the mean-value problem and 

\begin{equation}
\Pi := \left\{ \pi \in \mathbb{R}^6 \ \middle\vert
\begin{array}{l}
3\pi_1+2\pi_2+\pi_3+\pi_5 \leq -15\\
2\pi_1+5\pi_2+\pi_4+\pi_6 \leq -12 \\
\pi_1, \pi_2, \pi_5, \pi_5 \leq 0, \pi_3, \pi_4\geq 0,\\
\end{array}\right\}
\end{equation}

\begin{equation}
\Lambda := \left\{\lambda \in \mathbb{R}^6 \ \middle\vert
\begin{array}{l}
3\lambda_1+2\lambda_2+\lambda_3+\lambda_5 \leq 0\\
2\lambda_1+5\lambda_2+\lambda_4+\lambda_6 \leq 0 \\
-1\leq \lambda_i \leq 1 & \forall i=1, \dots, 6 \\
\lambda_1, \lambda_2, \lambda_5, \lambda_5 \leq 0, \lambda_3, \lambda_4\geq 0\\
\end{array}\right\}
\end{equation}
and $\theta = \sum_{s=1}^{4} Q(x,\xi^s)\times p_s$



In [1]:
import gurobipy as gp;
from gurobipy import GRB;
import numpy as np;

In [2]:
ξ = [[4,4], [4,8], [6,4], [6,8]];#scearios
N = len(ξ); #number of scenarios
p = [1/N for s in range(N)]; #probability distribution

nᵤ = 2; #number of first stage deceision variables 
nᵥ = 2; #number of second stage deceision variables 

cᵤ = [3,2]; #first stage cost vector
cᵥ = [15,12]; #second stage cost vector 

m = 6; #number of second stage constraints

#second stage optimality subproblem coefficient matrix
Wᵒ = [[-3,-2],
      [-2,-5],
      [1,0],
      [0,1],
      [-1,0],
      [0,-1]]; 

#second stage optimality subproblem coefficient matrix
Wᶠ = [[-3,-2, 1, -1],
      [-2,-5, 1, -1],
      [1,0, 1, -1],
      [0,1, 1, -1],
      [-1,0, 1, -1],
      [0,-1, 1, -1]]; 

ϵ = 1e-5; #tolarence parameter 

# Solve a mean value problem (MVP) to find a lower bound for θ:

In [3]:
ξ̄ = [5,6]; #averaged scenarios
R = len(ξ̄) #number of realizations
m_mvp = 2; #number of constraints

Ẑ = 0; #initial objective value

MVP = gp.Model("MVP");
MVP.modelSense = GRB.MINIMIZE;

x_mvp = {};
for j in range(nᵤ):
   x_mvp[j] = MVP.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, obj=cᵤ[j]);

y_mvp = {};
for j in range(nᵥ):
    y_mvp[j] = MVP.addVar(vtype=GRB.CONTINUOUS, lb = 0.8*ξ̄[j], ub = ξ̄[j], obj=-cᵥ[j]);
   

W = [[3,2],[2,5]];
for i in range(m_mvp):
    MVP.addConstr(sum(W[i][j]*y_mvp[j] for j in range(nᵥ))<= x_mvp[i]);
    
MVP.update();
MVP.setParam("OutputFlag", 0);

print("===============================================================");
MVP.optimize();
if MVP.status == GRB.OPTIMAL:
    Ẑ = MVP.objVal
    print('MVP objective: %g' % MVP.objVal);
    
    xval_MVP = {};
    for j in range(nᵤ):
        xval_MVP[j]=x_mvp[j].x
    print("x_mvp = ", xval_MVP)
    
    yval_MVP = {};
    for j in range(nᵥ):
        yval_MVP[j]=y_mvp[j].x;
else:
    print('No solution');
    print('status = ', MVP.status);

Using license file /Users/murwansiddig/gurobi.lic
Academic license - for non-commercial use only
MVP objective: 9.2
x_mvp =  {0: 24.6, 1: 34.0}


In [4]:
# Define master problem
Master_multi = gp.Model("Master_multi");
Master_multi.modelSense = GRB.MINIMIZE;


xᵐ = {};
for j in range(nᵤ):
   xᵐ[j] = Master_multi.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, obj=cᵤ[j]);

θᵐ = {};
for s in range(N):
    θᵐ[s] = Master_multi.addVar(vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = GRB.INFINITY, obj=p[s]);

#Lower bound for θ
Master_multi.addConstr(sum(cᵤ[j]*xᵐ[j] for j in range(nᵤ))+sum(θᵐ[s]*p[s] for s in range(N))>= Ẑ);

x̂ᵐ = np.zeros(nᵤ); #initialize the first stage decision values;
θ̂ᵐ = np.zeros(N); #initialize the second stage expected cost;
Master_multi.setParam("OutputFlag", 0);

In [5]:
# Define an optimality subproblem 
sub_opt = gp.Model("subprob_optimality");
sub_opt.modelSense = GRB.MINIMIZE; 

yᵒ = {};
for j in range(nᵥ):
    yᵒ[j] = sub_opt.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, obj=-cᵥ[j]);

Π = {};
for i in range(m):
    Π[i] = sub_opt.addConstr(sum(Wᵒ[i][j]*yᵒ[j] for j in range(nᵥ)) >= 0);
    
sub_opt.setParam("OutputFlag", 0);

In [6]:
# Define an feasibility subproblem 
sub_feas = gp.Model("subprob_feasibility");
sub_feas.modelSense = GRB.MINIMIZE; 

yᶠ = {};
v̅ = {};
v̲ = {}; 
for j in range(nᵥ):
    yᶠ[j] = sub_feas.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, obj=0);
    
for i in range(m):
    v̅[i] = sub_feas.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, obj=1);
    v̲[i] = sub_feas.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, obj=1);

Λ = {};
for i in range(m):
    Λ[i] = sub_feas.addConstr(sum(Wᶠ[i][j]*yᶠ[j] for j in range(nᵤ))+Wᶠ[i][2]*v̅[i]+Wᶠ[i][3]*v̲[i] >= 0);
sub_feas.setParam("OutputFlag", 0);

In [9]:
LBᵐ = 0;
UBᵐ = 1e10;

Ocuts = np.zeros(N);
Fcuts = 0;

iter = 0;
while (UBᵐ-LBᵐ)*1.0/UBᵐ > ϵ:
    iter+=1;
    
    # First solve the master problem, and get x value and θ
    x̂ᵐ = np.zeros(nᵤ); #initialize the first stage decision values;
    θ̂ᵐ = np.zeros(N); #initialize the second stage expected cost;
    Master_multi.optimize();
    for j in range(nᵤ):
        x̂ᵐ[j]=xᵐ[j].x #value of x
        
    for s in range(N):
        θ̂ᵐ[s] = θᵐ[s].x; #value of θ
    
    LBᵐ = Master_multi.objVal; #update the lower bound.
    
    
    π = np.zeros((N,m)); #initialize the optimality dual multipliers (assuming x̂ is feasible ∀ ξ)
    λ = np.zeros((N,m)); #initialize the feasibility dual multipliers (in case needed);
    Q = np.zeros(N); #initialize a vector for the optimality subproblem optimal objective value ∀ ξ,
    w = np.zeros(N); #initialize a vector for the feasibility subproblem optimal objective value ∀ ξ,
    FLAG = 0; #(0 if x̂ is feasible ∀ ξ, and 1 otherwise)
    #first two constraints are independent from ξ so update them seprately to avoid redundant calculation
    Π[0].setAttr(GRB.Attr.RHS, -x̂ᵐ[0]);
    Π[1].setAttr(GRB.Attr.RHS, -x̂ᵐ[1]);
    for s in range(N): 
        Π[2].setAttr(GRB.Attr.RHS, 0.8*ξ[s][0]);
        Π[3].setAttr(GRB.Attr.RHS, 0.8*ξ[s][1]);
        Π[4].setAttr(GRB.Attr.RHS, -ξ[s][0]);
        Π[5].setAttr(GRB.Attr.RHS, -ξ[s][1]);
        sub_opt.optimize();
        
        if sub_opt.status == GRB.OPTIMAL:
            Q[s] = -sub_opt.objVal;
            for i in range(m):
                π[s][i] = Π[i].pi;
        elif sub_opt.status == GRB.INFEASIBLE or sub_opt.status == GRB.INF_OR_UNBD:
            FLAG = 1;
            break;
        else:
            print('One of the scenario subproblems is neither optimal or infeasible');
            print('status = ', sub_opt.status);
            break
            exit(0)
    if FLAG == 1:
        D = {};
        Λ[0].setAttr(GRB.Attr.RHS, -x̂ᵐ[0]);
        Λ[1].setAttr(GRB.Attr.RHS, -x̂ᵐ[1]);
        for s in range(N): 
            Λ[2].setAttr(GRB.Attr.RHS, 0.8*ξ[s][0]);
            Λ[3].setAttr(GRB.Attr.RHS, 0.8*ξ[s][1]);
            Λ[4].setAttr(GRB.Attr.RHS, -ξ[s][0]);
            Λ[5].setAttr(GRB.Attr.RHS, -ξ[s][1]);
            sub_feas.optimize();#should always be optimal --  no need to check the status
            w[s] = sub_feas.objVal;
            D[s] = [-xᵐ[0], -xᵐ[1], 0.8*ξ[s][0], 0.8*ξ[s][1], -ξ[s][0], -ξ[s][1]];
            for i in range(m):
                λ[s][i] = Λ[i].pi;
            if w[s] > ϵ:
                #Add the feasibility cut  
                Master_multi.addConstr(0>= sum(D[s][i]*λ[s][i] for i in range(m)));
                Fcuts +=1;
                break
    else:
        θ̄ᵐ = {};
        for s in range(N):
            θ̄ᵐ[s]=-Q[s];
        if sum(cᵤ[j]*x̂ᵐ[j] for j in range(nᵤ))+sum(θ̄ᵐ[s]*p[s] for s in range(N)) < UBᵐ:
            UBᵐ = sum(cᵤ[j]*x̂ᵐ[j] for j in range(nᵤ))+sum(θ̄ᵐ[s]*p[s] for s in range(N));
        if (UBᵐ-LBᵐ)/max(1e-10,abs(LBᵐ)) < ϵ:
            break;
        else:
            for s in range(N):
                T = {};
                T[s] = [-xᵐ[0], -xᵐ[1], 0.8*ξ[s][0], 0.8*ξ[s][1], -ξ[s][0], -ξ[s][1]];
                if (θ̄ᵐ[s]-θ̂ᵐ[s])/max(1e-10,abs(θ̂ᵐ[s])) >= ϵ: 
                    #Add the optimality cut cut
                    Master_multi.addConstr(θᵐ[s]>=sum(T[s][i]*π[s][i] for i in range(m))); 
                    Ocuts[s] +=1;
    print("iter = ", iter);
    print("FLAG = ", FLAG)
    print("LB = ", LBᵐ);
    print("UB = ", UBᵐ);
    print("===============================================================");
    
print("#########################################################################################")
print("#########################################################################################")
print("iter = ", iter);
print("# of feasibility cuts added= ", Fcuts);
print("# of optimality cuts added = ", Ocuts);
print("LBᵐ = ", LBᵐ);
print("UBᵐ = ", UBᵐ);
print("x̂ᵐ = ", x̂ᵐ);
    

iter =  1
FLAG =  1
LB =  9.200000000000003
UB =  10000000000.0
iter =  2
FLAG =  1
LB =  9.200000000000003
UB =  10000000000.0
iter =  3
FLAG =  1
LB =  9.200000000000017
UB =  10000000000.0
iter =  4
FLAG =  1
LB =  9.200000000000017
UB =  10000000000.0
iter =  5
FLAG =  1
LB =  9.199999999999989
UB =  10000000000.0
iter =  6
FLAG =  1
LB =  9.199999999999989
UB =  10000000000.0
iter =  7
FLAG =  1
LB =  9.199999999999989
UB =  10000000000.0
iter =  8
FLAG =  1
LB =  9.199999999999989
UB =  10000000000.0
iter =  9
FLAG =  0
LB =  9.200000000000017
UB =  30.939999999999998
iter =  10
FLAG =  0
LB =  9.200000000000017
UB =  30.939999999999998
iter =  11
FLAG =  0
LB =  9.200000000000017
UB =  30.939999999999998
iter =  12
FLAG =  0
LB =  9.200000000000017
UB =  30.939999999999998
#########################################################################################
#########################################################################################
iter =  13
# of feasibility c